In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import random
import os
import requests
from config import weather_api_key

In [2]:
# Generate a new set of 1,500 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)


In [3]:
#Get the nearest city using the citipy module
cities = [citipy.nearest_city(lat_lng[0],lat_lng[1]) for lat_lng in lat_lngs ]


In [4]:
# Remove duplicate cities. Create the list, unique_cities
unique_cities = []
for city in set(cities):
    unique_cities.append(city)

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

<ul>
    <li>Perform an API call with the OpenWeatherMap.</li>
    <li>Retrieve the following information from the API call:
        <ul>
            <li>Latitude and longitude</li>
            <li>Maximum temperature</li>
            <li>Percent humidity</li>
            <li>Percent cloudiness</li>
            <li>Wind speed</li>
            <li>Weather description (e.g., clouds, fog, light rain, clear sky)</li>
            <li>Using a try-except block, in the try block if it is raining, get the amount of rainfall in inches for the last three hours. In the except block handle the KeyError if there is no rainfall and add 0 inches for the rainfall amount.</li>
            <li>Using a try-except block, in the try block if it is snowing, get the amount of snowfall in inches for the last three hours. In the except block handle the KeyError if there is no snowfall and add 0 inches for the snowfall amount.</li>
        </ul>
    </li>
</ul>
    


In [6]:
# Retrieve a list of city_weather json objects
# In order not to keep sending requests to the openweathermap service, this code segment captures the json responses
# and uses them to create a list of city_weather json objects to be processed in later segments. 
# The code checks each response for the ["coord"] key. If an exception is thrown, the next request is submitted. If 
# there is no exception, the json response is appended to the city_weather_ls list.

# A list of city_weather json objects
city_weather_ls = []

# Iterate through the unique_cities list and, for each, submit a get request to the openweatherdata service
for city in unique_cities:
    city_name = city.city_name
    city_url = url + "&q=" + city_name
    
    # Make a 'Get' request for the city weather and convert the response to a json object
    city_weather = requests.get(city_url).json()
    
    try:
        # If a city_weather object is returned, append it to the list, city_weather_ls
        city_coord = city_weather["coord"]  
        city_weather_ls.append(city_weather)
    except (KeyError):
        # If a city_weather object is not found, move on to the next city
        continue
    
    

In [7]:
# Function convert_mm_to_inches expects an argument in millimetres (mm) and returns the equivalent measurement in inches
# Arguments:
# mm - millimetre measurement
# Return:
# inch measurement

def convert_mm_to_inches(mm) -> float:
    return mm / 25.4

In [8]:
# Output weather_data to file, WeatherPy_challenge.csv
outfolder = "data"
outfile = "WeatherPy_challenge.csv"

# Create a city_weather list to hold the weather dictionaries for all cities
city_weather_list = []

# Iterate through list, city_weather_ls, and, for each item, capture the particulars like city name, country name, etc.
for city_weather in city_weather_ls:
    
    # Capture the weather parameters
    try:
        city_name = city_weather["name"]
        city_country = city_weather["sys"]["country"]
        city_latitude = city_weather["coord"]["lat"]
        city_longitude = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_cloudiness = city_weather["clouds"]["all"]
        city_wind_speed = city_weather["wind"]["speed"]
        city_weather_desc = city_weather["weather"][0]["description"]

    except (KeyError) as ke:
        # City not found. Move to next city.
        continue
    
    # Capture the amount of rainfall for the last 3 hours
    try:
        # If the json object contains a value for "rain"."3h", set the city_rain_fall to that value
        city_rain_fall = city_weather["rain"]["3h"] # millimetres
        city_rain_fall = convert_mm_to_inches(city_rain_fall) # Convert to inches
    except (KeyError):
        try:
            # If the json object contain a value for "snow"."1h", set the city_snam_fall to that value
            city_rain_fall = city_weather["rain"]["1h"] # millimetres
            city_rain_fall = convert_mm_to_inches(city_rain_fall)  # Convert to inches
        except (KeyError):
            # Otherwise, set city_rain_fall to 0
            city_rain_fall = 0
    
    # Capture the amount of snowfall for the last 3 hours
    try:
        # If the json object contains a value for "snow"."3h", set the city_snow_fall to that value
        city_snow_fall = city_weather["snow"]["3h"] # millimetres
        city_snow_fall = convert_mm_to_inches(city_snow_fall)  # Convert to inches
    except (KeyError):
        try:
            # If the json object contain a value for "snow"."1h", set the city_snam_fall to that value
            city_snow_fall = city_weather["snow"]["1h"] # millimetres
            city_snow_fall = convert_mm_to_inches(city_snow_fall)  # Convert to inches
        except (KeyError):
            # Otherwise, set city_snow_fall to 0
            city_snow_fall = 0
    
    # Append the dictionary for the current city to the city_weather_list dictionary list
    city_weather_list.append({"City":city_name,"Country":city_country,"Lat":city_latitude,"Lng":city_longitude,\
          "Max Temp":city_max_temp,"Humidity":city_humidity,"Cloudiness":city_cloudiness,\
          "Wind Speed":city_wind_speed,"Current Description":city_weather_desc,\
          "Rain (inches)":city_rain_fall,"Snow (inches)": city_snow_fall
         })
 
# Create city_weather_df dataframe from the city_weather_list dictionary list
city_weather_df = pd.DataFrame(city_weather_list)

# Format columns 
city_weather_df["Lat"] = city_weather_df["Lat"].map("{:.2f}".format)
city_weather_df["Lng"] = city_weather_df["Lng"].map("{:.2f}".format)
city_weather_df["Max Temp"] = city_weather_df["Max Temp"].map("{:.2f}".format)
city_weather_df["Rain (inches)"] = city_weather_df["Rain (inches)"].map("{:.1f}".format)
city_weather_df["Snow (inches)"] = city_weather_df["Snow (inches)"].map("{:.1f}".format)

# Output the city_weather_df dataframe to a csv file
city_weather_df.to_csv(os.path.join(outfolder,outfile),index=False)
city_weather_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Assiut,EG,27.18,31.18,89.60,29,0,17.22,clear sky,0.0,0.0
1,Salalah,OM,17.02,54.09,84.20,83,75,5.82,broken clouds,0.0,0.0
2,Nome,US,64.50,-165.41,42.01,93,90,8.05,overcast clouds,0.0,0.0
3,Grindavik,IS,63.84,-22.43,44.60,93,90,28.86,light rain,0.0,0.0
4,Hithadhoo,MV,-0.60,73.08,83.89,78,14,10.89,light rain,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
555,Cabo San Lucas,MX,22.89,-109.91,81.00,47,5,6.93,clear sky,0.0,0.0
556,Nhulunbuy,AU,-12.23,136.77,69.80,83,0,3.36,clear sky,0.0,0.0
557,Mumbwa,ZM,-14.98,27.06,66.63,33,9,2.64,clear sky,0.0,0.0
558,Hasaki,JP,35.73,140.83,64.40,88,75,6.93,broken clouds,0.0,0.0


Q. How many cities have recorded rainfall or snow?

In [10]:
# Read from csv because object in memory no longer usable
city_weather_df = pd.read_csv(os.path.join(outfolder,outfile)) 
f"The number of cities that have recorded rainfall or snow is " \
f"{city_weather_df[(city_weather_df['Rain (inches)'] > 0) | (city_weather_df['Snow (inches)'] > 0)].count()['City']}"

'The number of cities that have recorded rainfall or snow is 9'